# Import all the needed libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SCOPUS Dataset

### Import Dataset

In [2]:
scopus = pd.read_csv('SCOPUS Journals.csv') 

### Select Computer Science Subject Area

In [3]:
asjc_code = scopus['Scopus ASJC Code (Sub-subject Area)'].apply(str) 
scopus = scopus[asjc_code.apply(lambda x: x[:2]) == '17'] 

### Select Computer Science Journals

In [4]:
not_scopus_journal = scopus.loc[scopus['Type'] != 'j']
scopus = scopus.drop(not_scopus_journal.index, axis=0)

### Renumbering the index

In [5]:
scopus.index=range(len(scopus))

### Drop Unwanted Columns

In [6]:
scopus = scopus.drop(['Scopus Source ID', 'Scopus ASJC Code (Sub-subject Area)', 'Type', 'Top 10% (CiteScore Percentile)'], 
                     axis=1)

### Renaming Columns

In [7]:
scopus.rename(columns = {'Scopus Sub-Subject Area' : 'subject_area'}, inplace=True)

### Working on the Index Column

In [8]:
scopus['Index'] = ['Scopus']*len(scopus)

### Working on the Publisher Column

In [9]:
scopus['Publisher2'] = scopus['Publisher']
publisher = scopus['Publisher2']

#Springer
pub_springer = publisher[publisher.apply(lambda x: x[:8]) == 'Springer'].unique().tolist()
for i in publisher:
    if i in pub_springer:
        a = publisher.replace(i,"Springer", inplace=True)
        
#Elsevier
pub_elsevier = publisher[publisher.apply(lambda x: x[:8]) == 'Elsevier'].unique().tolist()
for i in publisher:
    if i in pub_elsevier:
        a = publisher.replace(i,"Elsevier", inplace=True)
        
#IEEE1
pub_ieee1 = publisher[publisher.apply(lambda x: x[:4]) == 'IEEE'].unique().tolist()
for i in publisher:
    if i in pub_ieee1:
        a = publisher.replace(i,"IEEE", inplace=True)

#IEEE2
pub_ieee2 = publisher[publisher.apply(lambda x: x[:49]) == 'Institute of Electrical and Electronics Engineers'].unique().tolist()
for i in publisher:
    if i in pub_ieee2:
        a = publisher.replace(i,"IEEE", inplace=True)

#Taylor and Francis
pub_taylor = publisher[publisher.apply(lambda x: x[:6]) == 'Taylor'].unique().tolist()
for i in publisher:
    if i in pub_taylor:
        a = publisher.replace(i,"Taylor and Francis", inplace=True)
    
#Inderscience
pub_indersci = publisher[publisher.apply(lambda x: x[:12]) == 'Inderscience'].unique().tolist()
for i in publisher:
    if i in pub_indersci:
        a = publisher.replace(i,"Inderscience", inplace=True)
        
#IGI
pub_igi = publisher[publisher.apply(lambda x: x[:3]) == 'IGI'].unique().tolist()
for i in publisher:
    if i in pub_igi:
        a = publisher.replace(i,"IGI Global", inplace=True)

#ACM
pub_acm = publisher[publisher.apply(lambda x: x[:41]) == 'Association for Computing Machinery (ACM)'].unique().tolist()
for i in publisher:
    if i in pub_acm:
        a = publisher.replace(i,"ACM", inplace=True)

#Others
main_publiser = publisher.value_counts().head(7)
main_publiser = main_publiser.index.tolist()
for i in publisher:
    if i not in main_publiser:
        a = publisher.replace(i, "Others", inplace=True)

### Working on the Percentile Column

In [10]:
scopus['Percentile2'] = scopus['Percentile']
percent = scopus['Percentile2']

for i in percent:
    if(i>=0 and i<=24):
        a = percent.replace(i, 400, inplace=True)

for i in percent:
    if(i>=25 and i<=49):
        a = percent.replace(i, 300, inplace=True)
        
for i in percent:
    if(i>=50 and i<=74):
        a = percent.replace(i, 200, inplace=True)

for i in percent:
    if(i>=75 and i<=99):
        a = percent.replace(i, 100, inplace=True)

### Working on the Open Access Column

In [11]:
scopus['Open Access2'] = scopus['Open Access']
open_access = scopus['Open Access2']

for i in open_access:
    if(i=="YES"):
        a = open_access.replace(i, "Yes", inplace=True)

for i in open_access:
    if(i=="NO"):
        a = open_access.replace(i, "No", inplace=True)

### Working on the Print ISSN Column

In [12]:
issn = scopus['Print ISSN']
issn_len = 8
for i in issn:
    if pd.isnull(i):
        new_issn = i
    elif len(i) != issn_len:
        z = issn_len - len(i)
        new_issn = str("0"*z)+(i)
        value = issn.replace(i, new_issn, inplace=True)
    else:
        new_issn = i
for i in issn:
    if pd.isnull(i):
        new_issn = i
    elif len(i) == 8:
        hyphen = i[:4] + '-' + i[4:]
        value = issn.replace(i, hyphen, inplace=True) 

### Working on the E-ISSN Column

In [13]:
eissn = scopus['E-ISSN']
eissn_len = 8
for i in eissn:
    if pd.isnull(i):
        new_eissn = i
    elif len(i) != eissn_len:
        z = eissn_len - len(i)
        new_eissn = str("0"*z)+(i)
        value = eissn.replace(i, new_eissn, inplace=True)
    else:
        new_eissn = i
for i in eissn:
    if pd.isnull(i):
        new_eissn = i
    elif len(i) == 8:
        hyphen = i[:4] + '-' + i[4:]
        value = eissn.replace(i, hyphen, inplace=True) 

### View Dataset

In [14]:
scopus.head(2)

,Title,Citation Count,Scholarly Output,Percent Cited,CiteScore,SNIP,SJR,subject_area,Percentile,RANK,...,Publisher,Open Access,Quartile,URL Scopus Source ID,Print ISSN,E-ISSN,Index,Publisher2,Percentile2,Open Access2
0,Computers in Education Journal,75,88,31.8,0.9,0.524,0.258,General Computer Science,24.0,168.0,...,American Society for Engineering Education,NO,4.0,https://www.scopus.com/sourceid/18388,1069-3769,NaN,Scopus,Others,400.0,No
1,Eurasip Journal on Embedded Systems,110,65,67.7,1.7,0.837,0.154,General Computer Science,50.0,111.0,...,Springer Nature,YES,2.0,https://www.scopus.com/sourceid/4000151614,1687-3955,1687-3963,Scopus,Springer,200.0,Yes


# SCIEISI Dataset

### Import Dataset

In [15]:
scieisi = pd.read_csv('SCIEISI Journals.csv')

### Remove null catergories rows

In [16]:
null_categories = scieisi.loc[scieisi['Web of Science Categories'].isnull()]
scieisi = scieisi.drop(null_categories.index, axis=0)

### Remove null print_issn rows

In [17]:
null_issn = scieisi.loc[scieisi['ISSN'].isnull()]
scieisi = scieisi.drop(null_issn.index, axis=0)

### Filter Computer Science Journals

In [18]:
scieisi['Catergories'] = [catergories.replace(' | ', ', ').split(', ') for catergories in scieisi['Web of Science Categories']]
scieisi = scieisi[scieisi['Catergories'].apply(lambda x: "Computer Science" in x)] 

### Renumbering the index

In [19]:
scieisi.index=range(len(scieisi))

### Drop unwanted columns

In [20]:
scieisi = scieisi.drop(['Publisher address', 'Languages', 'Catergories'], axis=1)

### Renaming columns

In [21]:
scieisi.rename(columns = {'Journal title': 'Title', 'ISSN': 'Print ISSN', 'eISSN': 'E-ISSN', 'Publisher name': 'Publisher',
                         'Web of Science Categories' : 'Subject Area'}, inplace=True)

### Working on the Index Column

In [22]:
scieisi['Index'] = ['SCIEISI']*len(scieisi)

### Working on the Publisher Column

In [23]:
scieisi['Publisher2'] = scieisi['Publisher'].str.lower()
publisher = scieisi['Publisher2'] 

#Springer
pub_springer = publisher[publisher.apply(lambda x: x[:8]) == 'springer'].unique().tolist()
for i in publisher:
    if i in pub_springer:
        a = publisher.replace(i,"Springer", inplace=True)
        
#Elsevier
pub_elsevier = publisher[publisher.apply(lambda x: x[:8]) == 'elsevier'].unique().tolist()
for i in publisher:
    if i in pub_elsevier:
        a = publisher.replace(i,"Elsevier", inplace=True)
        
#IEEE
pub_ieee1 = publisher[publisher.apply(lambda x: x[:4]) == 'ieee'].unique().tolist()
for i in publisher:
    if i in pub_ieee1:
        a = publisher.replace(i,"IEEE", inplace=True)

#Taylor and Francis
pub_taylor = publisher[publisher.apply(lambda x: x[:6]) == 'taylor'].unique().tolist()
for i in publisher:
    if i in pub_taylor:
        a = publisher.replace(i,"Taylor and Francis", inplace=True)
    
#Inderscience
pub_indersci = publisher[publisher.apply(lambda x: x[:12]) == 'inderscience'].unique().tolist()
for i in publisher:
    if i in pub_indersci:
        a = publisher.replace(i,"Inderscience", inplace=True)
        
#IGI
pub_igi = publisher[publisher.apply(lambda x: x[:3]) == 'igi'].unique().tolist()
for i in publisher:
    if i in pub_igi:
        a = publisher.replace(i,"IGI Global", inplace=True)

#ACM
pub_acm = publisher[publisher.apply(lambda x: x[:25]) == 'assoc computing machinery'].unique().tolist()
for i in publisher:
    if i in pub_acm:
        a = publisher.replace(i,"ACM", inplace=True)

#Others
main_publiser = publisher.value_counts().head(5)
main_publiser = main_publiser.index.tolist()
for i in publisher: 
    if i == "Inderscience":
        main_publiser.append(i)
    elif i == "IGI Global":
        main_publiser.append(i)
    elif i not in main_publiser:
        a = publisher.replace(i, "Others", inplace=True)

### Working on Subject Area Column

In [24]:
scieisi['Subject Area2'] = scieisi['Subject Area']
subject_area = scieisi['Subject Area2']

#Artificial Intelligence
sub_ai = subject_area[subject_area.apply(lambda x: "Artificial Intelligence" in x)].unique().tolist()
for i in subject_area:
    if i in sub_ai:
        a = subject_area.replace(i,"Artificial Intelligence", inplace=True)
        
#Information Systems
sub_is = subject_area[subject_area.apply(lambda x: "Information Systems" in x)].unique().tolist()
for i in subject_area:
    if i in sub_is:
        a = subject_area.replace(i,"Information Systems", inplace=True)

#Theory & Methods
sub_theory = subject_area[subject_area.apply(lambda x: "Theory & Methods" in x)].unique().tolist()
for i in subject_area:
    if i in sub_theory:
        a = subject_area.replace(i,"Computational Theory and Mathematics", inplace=True)
        
#Hardware & Architecture
sub_hardware = subject_area[subject_area.apply(lambda x: "Hardware & Architecture" in x)].unique().tolist()
for i in subject_area:
    if i in sub_hardware:
        a = subject_area.replace(i,"Hardware and Architecture", inplace=True)

#Software Engineering
sub_software = subject_area[subject_area.apply(lambda x: "Software Engineering" in x)].unique().tolist()
for i in subject_area:
    if i in sub_software:
        a = subject_area.replace(i,"Software", inplace=True)

#Cybernetics
sub_cyber = subject_area[subject_area.apply(lambda x: "Cybernetics" in x)].unique().tolist()
for i in subject_area:
    if i in sub_cyber:
        a = subject_area.replace(i,"Computer Science Applications", inplace=True)
        
#Interdisciplinary Applications 
sub_inter = subject_area[subject_area.apply(lambda x: "Interdisciplinary Applications" in x)].unique().tolist()
for i in subject_area:
    if i in sub_inter:
        a = subject_area.replace(i,"Computer Science Applications", inplace=True)

### View Dataset

In [25]:
scieisi.head(2)

,Title,Print ISSN,E-ISSN,Publisher,Subject Area,Index,Publisher2,Subject Area2
0,ACM COMPUTING SURVEYS,0360-0300,1557-7341,ASSOC COMPUTING MACHINERY,"Computer Science, Theory & Methods",SCIEISI,ACM,Computational Theory and Mathematics
1,ACM JOURNAL ON COMPUTING AND CULTURAL HERITAGE,1556-4673,1556-4711,ASSOC COMPUTING MACHINERY,"Computer Science, Interdisciplinary Applications",SCIEISI,ACM,Computer Science Applications


# Merging Dataset

In [26]:
scopus_issn = scopus['Print ISSN']
scieisi_issn = scieisi['Print ISSN']

present_journals = []
for i in scieisi_issn:
    if i in scopus_issn.unique().tolist():
        present_journals.append(i)
        scieisi.drop(scieisi[scieisi['Print ISSN'] == i].index, inplace=True)
        scopus.loc[scopus_issn == i, 'Index'] = "Scopus & SCIEISI"     

In [27]:
scopus.head(3)

,Title,Citation Count,Scholarly Output,Percent Cited,CiteScore,SNIP,SJR,subject_area,Percentile,RANK,...,Publisher,Open Access,Quartile,URL Scopus Source ID,Print ISSN,E-ISSN,Index,Publisher2,Percentile2,Open Access2
0,Computers in Education Journal,75,88,31.8,0.9,0.524,0.258,General Computer Science,24.0,168.0,...,American Society for Engineering Education,NO,4.0,https://www.scopus.com/sourceid/18388,1069-3769,NaN,Scopus,Others,400.0,No
1,Eurasip Journal on Embedded Systems,110,65,67.7,1.7,0.837,0.154,General Computer Science,50.0,111.0,...,Springer Nature,YES,2.0,https://www.scopus.com/sourceid/4000151614,1687-3955,1687-3963,Scopus,Springer,200.0,Yes
2,International Journal of Information Technolog...,43,40,42.5,1.1,0.290,0.118,General Computer Science,31.0,152.0,...,IGI Global Publishing,NO,3.0,https://www.scopus.com/sourceid/21100241217,1935-570X,1935-5718,Scopus,IGI Global,300.0,No


In [28]:
scieisi.head(3)

,Title,Print ISSN,E-ISSN,Publisher,Subject Area,Index,Publisher2,Subject Area2
4,ACM TRANSACTIONS ON ALGORITHMS,1549-6325,1549-6333,ASSOC COMPUTING MACHINERY,"Mathematics, Applied | Computer Science, Theor...",SCIEISI,ACM,Computational Theory and Mathematics
36,ADVANCES IN COMPUTERS,0065-2458,NaN,ELSEVIER ACADEMIC PRESS INC,"Computer Science, Hardware & Architecture | Co...",SCIEISI,Elsevier,Hardware and Architecture
46,APPLICABLE ALGEBRA IN ENGINEERING COMMUNICATIO...,0938-1279,1432-0622,SPRINGER,"Computer Science, Interdisciplinary Applicatio...",SCIEISI,Springer,Computational Theory and Mathematics


### Export Dataset

In [29]:
scopus.to_csv('Ranking Journals.csv', index=False)